In [0]:
import pandas as pd
import numpy as np

In [0]:
#Funcao responsavel por calcular a distancia euclidiana entre os usuarios.
def distancia_de_vetores(a,b):
  return np.linalg.norm(a - b)

In [0]:
#funcao responsavel por fornecer as notas que um usuario deu para um serviço
def notas_do_usuario(usuario, numero_de_recomendacoes = 100):
  if vendas.query("Cliente==%d" % usuario).dropna().shape[0] == 0:
    recomendacao_novo =  vendas.groupby('Serviço').mean()[['nota']]
    recomendacao_novo =  recomendacao_novo.sort_values('nota', ascending = False)
    return recomendacao_novo.head(4)

  notas_do_usuario = vendas.query("Cliente==%d" % usuario)
  notas_do_usuario = notas_do_usuario[["Serviço", "nota"]].set_index("Serviço")
  return notas_do_usuario

In [0]:
#Funcao para calcular a distancia entre um usuario e outro
#como meu banco de dados tem poucos clientes, coloquei que os usuarios precisam ter no minimo 1 semelhanca
def distancia_de_usuarios(usuario1, usuario2, minimo = 0):
  notas1 = notas_do_usuario(usuario1)
  notas2 = notas_do_usuario(usuario2)
  diferencas = notas1.join(notas2,lsuffix = '_esquerda', rsuffix = '_direita').dropna()
  
  if(len(diferencas) < minimo):
    return None
  
  distancia =  distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
  
  return [usuario1, usuario2, distancia]

In [0]:
#Funcao responsavel para calcular a distancia entre usuario e todos os usuarios, ou parte, isso pode ser alterado
def distancia_de_todos(voce, numero_de_usuarios_a_analisar = None):
  todos_os_usuarios = vendas['Cliente'].unique()
  if numero_de_usuarios_a_analisar:
    todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar]
  distancias = [distancia_de_usuarios(voce, cliente) for cliente in todos_os_usuarios]
  distancias = list(filter(None,distancias))
  distancias = pd.DataFrame(distancias, columns = ['voce','outra_pessoa','distancia'])
  return distancias

In [0]:
#essa funcao vai receber todas as distancias euclidianas e devolver os usuarios mais proximos de "voce" mais proximos.
def mais_proximos_de(voce, n_mais_proximos= 10 ,numero_de_usuarios_a_analisar = None):
  distancias = distancia_de_todos(voce, numero_de_usuarios_a_analisar)
  distancias = distancias.sort_values("distancia")
  distancias = distancias.set_index("outra_pessoa").drop(voce)
  return distancias.head(n_mais_proximos)

In [0]:
#essa é a funcao responsavel por devolver os servicos recomendados para um usuario
#ela recebe como parametro "voce", que é a Id do usuario.
def sugere_para(voce, 
                n_mais_proximos = 4, 
                numero_de_usuarios_a_analisar = None,
):
  notas_de_voce = notas_do_usuario(voce)
  servicos_ja_usados = notas_de_voce.index
  
  similares = mais_proximos_de(voce,
                               n_mais_proximos = n_mais_proximos,
                               numero_de_usuarios_a_analisar = 
                               numero_de_usuarios_a_analisar,
                               )
  
  usuarios_similares = similares.index
  notas_dos_similares = vendas.set_index('Cliente').loc[usuarios_similares]

  recomendacoes = notas_dos_similares.groupby('Serviço').mean()[['nota']]
  recomendacoes = recomendacoes.sort_values("nota",ascending = False)
  
  return recomendacoes.head(4).index
  

In [0]:
def recomendados(numero_de_recomendacoes = 4):
  x = [cliente for cliente in vendas['Cliente'].unique()]
  return [sugere_para(cliente,numero_de_recomendacoes) for cliente in x]

In [0]:
vendas = pd.read_csv("vendas15.csv")
recomendados()

[Int64Index([1, 2, 7, 12], dtype='int64', name='Serviço'),
 Int64Index([7, 12, 4, 6], dtype='int64', name='Serviço'),
 Int64Index([2, 7, 9, 12], dtype='int64', name='Serviço'),
 Int64Index([2, 7, 12, 1], dtype='int64', name='Serviço'),
 Int64Index([2, 7, 12, 1], dtype='int64', name='Serviço')]